In [1]:
import os
import importlib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image as PILImage
from random import uniform

In [2]:
# --- Parámetros generales ---
ROBOT_RADIO = 10
PROYECTIL_RADIO = 2
PROYECTIL_VEL = 60
ROBOT_VEL = 15
ARENA_ANCHO = 200
ARENA_ALTO = 200

In [3]:
# ============================================================
#  CLASES PRINCIPALES
# ============================================================

class Robot:
    """Representa un robot dentro de la arena."""

    def __init__(self, nombre, clase_control, x, y, arena):
        self.nombre = nombre
        self.control = clase_control(self)
        self.arena = arena
        self.x = x
        self.y = y
        self.angulo = np.random.uniform(0, 2*np.pi)
        self.salud = 100
        self.tiempo_disparo = 0

    def puede_disparar(self):
        return self.tiempo_disparo <= 0

    def actualizar_cooldown(self, dt):
        self.tiempo_disparo = max(0, self.tiempo_disparo - dt)


class Proyectil:
    """Representa un proyectil en vuelo."""

    def __init__(self, x, y, angulo, propietario):
        self.x = x
        self.y = y
        self.angulo = angulo
        self.propietario = propietario
        self.vivo = True


class Obstaculo:
    """Representa un obstáculo circular."""
    def __init__(self, x, y, r):
        self.x = x
        self.y = y
        self.r = r


class Arena:
    """La arena del combate."""

    def __init__(self):
        self.robots = []
        self.proyectiles = []
        self.obstaculos = []
        self.tiempo = 0

    def agregar_robot(self, robot):
        self.robots.append(robot)

    def agregar_obstaculo(self, obstaculo):
        self.obstaculos.append(obstaculo)

    def paso(self, dt):
        """Avanza un paso de simulación."""
        self.tiempo += dt

        for r in self.robots:
            if r.salud > 0:
                r.control.decision()
                self.mover_robot(r, dt)
                r.actualizar_cooldown(dt)

        for p in self.proyectiles:
            self.mover_proyectil(p, dt)

        self.detectar_impactos()

    def mover_robot(self, robot, dt):
        dx = np.cos(robot.angulo) * ROBOT_VEL * dt
        dy = np.sin(robot.angulo) * ROBOT_VEL * dt
        nuevo_x = np.clip(robot.x + dx, ROBOT_RADIO, ARENA_ANCHO - ROBOT_RADIO)
        nuevo_y = np.clip(robot.y + dy, ROBOT_RADIO, ARENA_ALTO - ROBOT_RADIO)

        # Evitar obstáculos
        for o in self.obstaculos:
            if np.hypot(nuevo_x - o.x, nuevo_y - o.y) < ROBOT_RADIO + o.r:
                return  # bloquea movimiento

        robot.x = nuevo_x
        robot.y = nuevo_y

    def mover_proyectil(self, proyectil, dt):
        proyectil.x += np.cos(proyectil.angulo) * PROYECTIL_VEL * dt
        proyectil.y += np.sin(proyectil.angulo) * PROYECTIL_VEL * dt

        # Colisión con bordes
        if not (0 <= proyectil.x <= ARENA_ANCHO and 0 <= proyectil.y <= ARENA_ALTO):
            proyectil.vivo = False
            return

        # Colisión con obstáculos
        for o in self.obstaculos:
            if np.hypot(proyectil.x - o.x, proyectil.y - o.y) < PROYECTIL_RADIO + o.r:
                proyectil.vivo = False
                return

    def detectar_impactos(self):
        for p in list(self.proyectiles):
            for r in self.robots:
                if r is p.propietario or r.salud <= 0:
                    continue
                if np.hypot(p.x - r.x, p.y - r.y) < ROBOT_RADIO:
                    r.salud -= 25
                    p.vivo = False
        self.proyectiles = [p for p in self.proyectiles if p.vivo]

    def disparar(self, robot):
        """Dispara un proyectil si puede."""
        if robot.puede_disparar():
            nuevo = Proyectil(robot.x, robot.y, robot.angulo, robot)
            self.proyectiles.append(nuevo)
            robot.tiempo_disparo = 0.8

    def render(self, ax):
        """Dibuja el estado de la arena."""
        ax.cla()
        ax.set_xlim(0, ARENA_ANCHO)
        ax.set_ylim(0, ARENA_ALTO)
        ax.set_aspect('equal')
        ax.set_title(f"Tiempo: {self.tiempo:.1f}s")

        # Obstáculos
        for o in self.obstaculos:
            ax.add_patch(plt.Circle((o.x, o.y), o.r, color='gray', alpha=0.4))

        # Robots
        for r in self.robots:
            color = "blue" if r.salud > 0 else "red"
            ax.add_patch(plt.Circle((r.x, r.y), ROBOT_RADIO, color=color, alpha=0.6))
            ax.arrow(r.x, r.y, np.cos(r.angulo)*15, np.sin(r.angulo)*15,
                     head_width=5, fc='black', ec='black')

        # Proyectiles
        for p in self.proyectiles:
            ax.add_patch(plt.Circle((p.x, p.y), PROYECTIL_RADIO, color="orange"))


In [4]:
# ============================================================
#  FUNCIONES AUXILIARES
# ============================================================

def cargar_bots_desde_carpeta(carpeta="bots"):
    """Carga todos los bots de la carpeta dada."""
    bots = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".py") and archivo != "__init__.py":
            nombre_modulo = archivo[:-3]
            modulo = importlib.import_module(f"{carpeta}.{nombre_modulo}")
            if hasattr(modulo, "Bot"):
                bots.append((nombre_modulo, modulo.Bot))
    return bots


def guardar_gif(arena, nombre_archivo="combate.gif", pasos=400, dt=0.1, fps=20):
    """Genera un GIF de la simulación."""
    matplotlib.use("Agg")
    fig, ax = plt.subplots(figsize=(6, 6))
    frames = []

    for i in range(pasos):
        arena.paso(dt)
        arena.render(ax)
        fig.canvas.draw()

        image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        frames.append(PILImage.fromarray(image[..., :3]))

        vivos = [r for r in arena.robots if r.salud > 0]
        if len(vivos) <= 1:
            break

    plt.close(fig)
    frames[0].save(nombre_archivo, save_all=True, append_images=frames[1:], duration=int(1000/fps), loop=0)
    return nombre_archivo


def ejecutar_combate():
    """Ejecuta la simulación completa y devuelve ganador y GIF."""
    arena = Arena()
    bots_disponibles = cargar_bots_desde_carpeta()

    if not bots_disponibles:
        raise RuntimeError("No se encontraron bots en la carpeta 'bots'.")

    # Posicionar robots
    for i, (nombre, clase_bot) in enumerate(bots_disponibles):
        x = 50 + i*100
        y = uniform(50, ARENA_ALTO-50)
        arena.agregar_robot(Robot(nombre, clase_bot, x, y, arena))

    # Obstáculos aleatorios
    for _ in range(5):
        x = uniform(40, ARENA_ANCHO-40)
        y = uniform(40, ARENA_ALTO-40)
        r = uniform(10, 25)
        arena.agregar_obstaculo(Obstaculo(x, y, r))

    archivo_gif = guardar_gif(arena)
    robots_vivos = sorted(arena.robots, key=lambda r: r.salud, reverse=True)
    ganador = robots_vivos[0].nombre if robots_vivos else "Nadie"

    print(f"Ganador: {ganador}")
    print(f"GIF guardado en: {archivo_gif}")
    return ganador, archivo_gif

In [5]:
ganador, gif = ejecutar_combate()

AttributeError: 'Robot' object has no attribute 'mover'